# Cross Validation 1 

Using CV to test an algorithm

We will use credit card default data from https://www.kaggle.com/datasets/uciml/default-of-credit-card-clients-dataset


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:,.2f}'.format

## Step-1: Download Data

In [2]:
import os
import urllib.request

data_url = 'https://raw.githubusercontent.com/elephantscale/datasets/master/credit-card-default/default.csv'
data_location = os.path.basename(data_url)

if not os.path.exists (data_location):
    print("Downloading : ", data_url)
    urllib.request.urlretrieve(data_url, data_location)
print('data_location:', data_location)

data_location: default.csv


In [3]:
data = pd.read_csv(data_location)
data.sample(10)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
29494,29495,320000,1,1,2,34,0,0,0,0,...,104795,113156,121465,19246,11550,11688,12122,12156,2437,0
24093,24094,210000,1,2,2,28,2,0,0,-1,...,205759,151849,150221,10000,2572,213509,6022,5817,4048,1
12515,12516,30000,1,2,2,28,-1,0,0,0,...,28720,780,0,3000,5000,2000,780,0,0,1
14666,14667,320000,2,2,1,43,-1,-1,-1,-1,...,5315,33059,908,30161,82271,5315,33059,908,6382,0
15930,15931,310000,2,2,1,29,-2,-2,-2,-2,...,3024,0,0,1062,8805,3087,0,0,3240,0
20295,20296,260000,2,3,1,52,1,2,0,0,...,27022,28402,800,0,10000,11264,1580,2,1098,1
3629,3630,120000,1,3,1,38,0,0,0,0,...,59862,51172,52533,1988,18000,1857,1723,2075,1500,0
2465,2466,100000,2,3,1,29,4,3,2,0,...,114431,116766,95961,0,0,0,2335,102075,0,1
27767,27768,50000,2,2,1,25,-1,-1,-1,2,...,5797,2413,33353,2463,8514,0,0,33353,5868,0
8857,8858,190000,1,1,2,29,1,-1,2,2,...,66,0,4433,2214,0,0,66,4433,0,1


## Step-2: EDA

In [4]:
## Check data skew
data['default'].value_counts()

0    23364
1     6636
Name: default, dtype: int64

In [5]:
data['default'].value_counts(normalize=True)

0   0.78
1   0.22
Name: default, dtype: float64

## Step-3: Clean up 

TODO

## Step-4: Shape data

In [6]:
label_col = 'default'

feature_columns = data.columns
## TODO : drop 'ID' and 'default' columns
feature_columns = feature_columns.drop (['ID', 'default'])
#print (feature_columns)

In [7]:
X = data[feature_columns]
y = data[[label_col]]

print (X.shape)
print (y.shape)

(30000, 23)
(30000, 1)


## Step-5: Cross Validation

In [8]:
%%time 

from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# algo = LogisticRegression(max_iter=500)
# algo = KNeighborsClassifier()
algo =  SVC()
# algo = RandomForestClassifier()

cv_results = cross_validate(estimator= algo, 
                            X=X, 
                            y=y, 
                            cv=5, 
                            scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
                            return_train_score=True,
                            return_estimator=True,
                            n_jobs= -1)

CPU times: user 85.5 ms, sys: 116 ms, total: 201 ms
Wall time: 1min 15s


In [9]:
cv_results

{'fit_time': array([22.80542254, 22.54385734, 23.00198817, 22.1343534 , 23.67215014]),
 'score_time': array([ 9.77455354,  9.62802744,  9.80052114,  9.76690412, 10.87371063]),
 'estimator': [SVC(), SVC(), SVC(), SVC(), SVC()],
 'test_accuracy': array([0.77866667, 0.77883333, 0.77883333, 0.77883333, 0.77883333]),
 'train_accuracy': array([0.77895833, 0.77895833, 0.77904167, 0.77908333, 0.77904167]),
 'test_precision': array([0., 0., 0., 0., 0.]),
 'train_precision': array([1., 1., 1., 1., 1.]),
 'test_recall': array([0., 0., 0., 0., 0.]),
 'train_recall': array([0.00056518, 0.00075344, 0.00113016, 0.00131852, 0.00113016]),
 'test_f1': array([0., 0., 0., 0., 0.]),
 'train_f1': array([0.00112973, 0.00150574, 0.00225776, 0.00263356, 0.00225776]),
 'test_roc_auc': array([0.55623962, 0.56348532, 0.54311699, 0.59259175, 0.52557719]),
 'train_roc_auc': array([0.59984213, 0.58304627, 0.59764864, 0.61565551, 0.5695038 ])}

In [10]:
from numpy import mean

print ("accuracy range = ",  min(cv_results['test_accuracy']),  " to ", max(cv_results['test_accuracy']))
print ("average accuracy : ", mean(cv_results['test_accuracy']))

accuracy range =  0.7786666666666666  to  0.7788333333333334
average accuracy :  0.7788000000000002


/home/sujee/anaconda3/envs/pyspark/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sujee/anaconda3/envs/pyspark/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sujee/anaconda3/envs/pyspark/lib/python3.8/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sujee/anaconda3/envs/pyspark/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMet